Code for the computations of RMS of \kappa^s in the linear case

In [ ]:
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import camb
from camb import model, initialpower
from scipy import integrate
from scipy.special import spherical_jn
import astropy.units as unit
from scipy.special import sici
import pyfftlog
from scipy.interpolate import interp1d
import h5py
# Constants in SI
eV2Joule = 1.6021*10**(-19)
hbar = 6.62 * 10**(-34) / (2* np.pi)
pc2meter = 3.086 * 10**(16)
arcsec2rad = 4.84814 * 10**(-6)
clight = 3*10**8
G_const = 6.67 * 10**(-11)
m_sun = 1.989 * 10**(30)

# Cosmology
h0 = 0.674
Om = 0.122/h0**2
Ob = 0.022/h0**2
OL = 1 - Om - Ob

#Set up a new set of parameters for CAMB
pars = camb.CAMBparams()
#This function sets up CosmoMC-like settings, with one massive neutrino and helium set using BBN consistency
pars.set_cosmology(H0=67.4, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0, tau=0.06)
pars.InitPower.set_params(ns=0.965, r=0)
# Linear power spectrum
kmax = 1000.
zmax = 5
pars.NonLinear = model.NonLinear_none

results= camb.get_background(pars)

#Get the matter power spectrum interpolation object (based on RectBivariateSpline).
PK_lin = camb.get_matter_power_interpolator(pars, nonlinear=False,
    hubble_units=False, k_hunit=False, kmax=kmax,
    zmax=0.1)
def PK_lin_func(k):
    return PK_lin.P(0,k)

def D_plus(z):
    Ez2 = OL + Om * (1 + z)**3
    Omz = Om * (1 + z)**3 / Ez2
    OLz = OL / Ez2
    normalization = 5/2 * Om *( Om**(4./7) - OL + (1 + Om/2)*( 1 + OL/70 ))**(-1)
    return 5/2 * Omz *( Omz**(4./7) - OLz + (1 + Omz/2)*( 1 + OLz/70 ))**(-1)/(1 + z)/ normalization

def qq_factor(w, deltaw, z1, z2, z3, z4):
    w_1 = results.comoving_radial_distance(z1)
    w_2 = results.comoving_radial_distance(z2)
    w_3 = results.comoving_radial_distance(z3)
    w_4 = results.comoving_radial_distance(z4)
    factor = (w_2 - w) * (w - w_1)/(w_2 - w_1) *(w_4 - w - deltaw) * (w + deltaw - w_3)/(w_4 - w_3)
    return factor

def integrand_wwk(w, deltaw, k, z1, z2, z3, z4):
    w_3 = results.comoving_radial_distance(z3)
    w_4 = results.comoving_radial_distance(z4)
    if w + deltaw > w_4 or w + deltaw < w_3:
        return 0
    else:
        qq = qq_factor(w, deltaw, z1, z2, z3, z4)
        z_dummy = results.redshift_at_comoving_radial_distance(w)
        z_dummy2 = results.redshift_at_comoving_radial_distance(w + deltaw)
        power_spect = PK_lin_func(k) * D_plus(z_dummy) * D_plus(z_dummy2)
        return power_spect * qq

def integrand_wk(k, deltaw, z1, z2, z3, z4):
    w_1 = results.comoving_radial_distance(z1)
    w_2 = results.comoving_radial_distance(z2)
    result = integrate.quad(integrand_wwk, w_1, w_2, args=(deltaw, k, z1, z2, z3, z4,))[0]
    return result

def integrand_deltaw(deltaw, z1, z2, z3, z4):
    if np.abs(deltaw) < 1.10e-3:
        integr_check = lambda ki: ki**2 *integrand_wk(ki, deltaw, z1, z2, z3, z4)* spherical_jn(0,ki*deltaw)
        result = integrate.quad(integr_check, 0, kmax)[0]
        return result
    else:
        logkmin = -4
        logkmax = 3
        n = 128 # Number of points (Max 4096)
        mu = 0.5 # Order mu of Bessel function
        q = 0 # Bias exponent: q = 0 is unbiased
        rr = 1 # w_2 - w_1 # Sensible approximate choice of k_c r_c
        rropt = 1 # Tell fhti to change r to low-ringing value WARNING: rropt = 3 will fail, as interaction is not supported
        tdir = 1 # Forward transform (changed from dir to tdir, as dir is a python fct)
        logkc = (logkmin + logkmax)/2 # Central point log10(k_c) of periodic interval
        nc = (n + 1)/2.0 # Central index (1/2 integral if n is even)
        dlogk = (logkmax - logkmin)/n # Log-spacing of points
        dlnk = dlogk*np.log(10.0)
        k = 10**(logkc + (np.arange(1, n+1) - nc)*dlogk)
        integrand = [ki**2 *integrand_wk(ki, deltaw, z1, z2, z3, z4) * np.sqrt(np.pi/(2*ki * np.abs(deltaw**3))) for ki in k]
        integrand = np.array(integrand)
        rr, xsave = pyfftlog.fhti(n, mu, dlnk, q, rr, rropt)

        logrc = np.log10(rr) - logkc
        # rk = r_c/k_c
        kk = 10**(logkc - logrc)
        result = pyfftlog.fht(integrand.copy(), xsave, tdir)
        delta_w_interp = 10**(logrc + (np.arange(1, n+1) - nc)*dlogk)
        result_interp = interp1d(delta_w_interp, result)
        return result_interp(np.abs(deltaw))

# Cosmological prefactor
cosmo_factor = 2/(2*np.pi)**2 * (3 * (100*h0)**2 * Om /2 /clight**2 * 10**6)**2

def kappa_ext(z1, z2, z3, z4):
    deltaw_array = np.logspace(-5, 3.5, 50)
    array_int_deltaw = [integrand_deltaw(wi, z1,z2,z3,z4) for wi in deltaw_array]
    k_variance = np.sqrt(cosmo_factor * 2*integrate.simps(array_int_deltaw, deltaw_array))
    return k_variance

compute_array = False
if compute_array == True:
    z_s1 = np.linspace(0.5, 3.8, 8)
    array_kappa_s = np.zeros((len(z_s1), len(z_s1)))
    for i in range(len(z_s1)):
        for j in range(len(z_s1)):
            if j>=i:
                kappa_s = kappa_ext(0, z_s1[i], 0, z_s1[j])
                print(kappa_s, z_s1[i], z_s1[j])
                array_kappa_s[i,j] = kappa_s
            else:
                array_kappa_s[i,j] = array_kappa_s[j,i]
    print(array_kappa_s)
    file_name = 'data_ext_kappa/data_kappa_s_linear.h5'
    h5file = h5py.File(file_name, 'w')
    h5file.create_dataset("dataset_kappa", data=array_kappa_s)
    h5file.create_dataset("dataset_zs1", data=z_s1)
    h5file.close()
else:
    file_name = 'data_ext_kappa/data_kappa_s_linear.h5'
    h5file = h5py.File(file_name, 'r')
    array_kappa = h5file['dataset_kappa'][:]
    z_s1 = h5file['dataset_zs1'][:]
    h5file.close()

def delta_kappa_ext(z_s1, z_s2):
    kappa_tot = np.sqrt(kappa_ext(0,z_s1,0,z_s1)**2 + kappa_ext(0,z_s2,0,z_s2)**2 - 2*kappa_ext(0,z_s1,0,z_s2)**2)
    return kappa_tot
